<a href="https://colab.research.google.com/github/gokulgobinathan/MACHINE-LEARNING/blob/main/space_titanic_using_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn import svm
import  numpy as np

In [ ]:
train_df=pd.read_csv('/content/train (1).csv')
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [ ]:
train_df.interpolate(method ='linear', limit_direction ='forward', inplace=True)
train_df.fillna(method='bfill',inplace=True)
train_df.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [ ]:
train_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)
train_df[['Group','ID']]=train_df['PassengerId'].str.split('_',expand=True)
train_df['HomePlanet']=train_df['HomePlanet'].map({'Europa':1,'Earth':2,'Mars':3}).astype('int8')
train_df['DeckSide']=train_df['DeckSide'].map({'P':1,'S':2}).astype('int8')
train_df['DeckName']=train_df['DeckName'].map({'B':1, 'F':2, 'A':3, 'G':4, 'E':5, 'D':6, 'C':7, 'T':8}).astype('int8')
train_df['Destination']=train_df['Destination'].map({'TRAPPIST-1e':1, 'PSO J318.5-22':2, '55 Cancri e':3}).astype('int8')
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,DeckName,DeckNumber,DeckSide,Group,ID
0,0001_01,1,False,B/0/P,1,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,1,0,1,0001,01
1,0002_01,2,False,F/0/S,1,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,2,0,2,0002,01
2,0003_01,1,False,A/0/S,1,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,3,0,2,0003,01
3,0003_02,1,False,A/0/S,1,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,3,0,2,0003,02
4,0004_01,2,False,F/1/S,1,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,2,1,2,0004,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,1,False,A/98/P,3,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,3,98,1,9276,01
8689,9278_01,2,True,G/1499/S,2,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,4,1499,2,9278,01
8690,9279_01,2,False,G/1500/S,1,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,4,1500,2,9279,01
8691,9280_01,1,False,E/608/S,3,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,5,608,2,9280,01


In [ ]:
train_df['CryoSleep']=train_df['CryoSleep'].astype(int)
train_df['VIP']=train_df["VIP"].astype(int)
train_df['Transported']=train_df['Transported'].astype(int)

In [ ]:
x_train=train_df[['Group','HomePlanet','CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','DeckName','DeckNumber','DeckSide']]
y_train=train_df['Transported']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( x_train, y_train, test_size=0.2, random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:

y_test_prediction = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, y_test_prediction)

print("Accuracy:", accuracy)

Accuracy: 0.8562392179413456


In [ ]:
test_path=pd.read_csv('/content/test (1).csv')

In [ ]:
test_df=pd.DataFrame(test_path)
test_df.interpolate(method ='linear', inplace=True)
test_df.fillna(method='bfill',inplace=True)

In [ ]:
test_df[['DeckName','DeckNumber','DeckSide']]=test_df['Cabin'].str.split('/',expand=True)
test_df[['Group','ID']]=test_df['PassengerId'].str.split('_',expand=True)
test_df['HomePlanet']=test_df['HomePlanet'].map({'Europa':1,'Earth':2,'Mars':3}).astype('int8')
test_df['DeckSide']=test_df['DeckSide'].map({'P':1,'S':2}).astype('int8')
test_df['DeckName']=test_df['DeckName'].map({'B':1, 'F':2, 'A':3, 'G':4, 'E':5, 'D':6, 'C':7, 'T':8}).astype('int8')
test_df['Destination']=test_df['Destination'].map({'TRAPPIST-1e':1, 'PSO J318.5-22':2, '55 Cancri e':3}).astype('int8')
test_df['CryoSleep']=test_df['CryoSleep'].astype(int)
test_df['VIP']=test_df["VIP"].astype(int)

In [ ]:
x_test = test_df[['Group', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'DeckName', 'DeckNumber', 'DeckSide']]
passengers = test_df['PassengerId']

# Make predictions using the trained KNN classifier on the test data
y_test_prediction = knn.predict(x_test)

# Create the final DataFrame
final = pd.DataFrame({'PassengerId': passengers, 'Transported': y_test_prediction.astype(bool)})
final.head(20)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
5,0027_01,True
6,0029_01,True
7,0032_01,True
8,0032_02,True
9,0033_01,False


In [ ]:
final.to_csv('submission.csv',index=False)
sub=pd.read_csv('./submission.csv')
sub.head(25)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
5,0027_01,True
6,0029_01,True
7,0032_01,True
8,0032_02,True
9,0033_01,False


In [ ]:
from  google.colab import files
files.download('submission.csv')